In [1]:
from camping import storage
from camping import ridb_data
import pandas as pd
from geopy.distance import vincenty
import itertools
import config
import numpy as np
from classes import Utilities

In [2]:
request_url = "http://" + config.LAMP_IP + '/ridb_mock.json'
df_ridb = pd.read_json(request_url)

In [73]:
ridb_slice = df_ridb[['FacilityName','FacilityLatitude','FacilityLongitude','LastUpdatedDate']].iloc[12:18]
ridb_slice.shape

(5, 4)

In [22]:
#ridb_slice = ridb_slice.set_index('FacilityName')

In [64]:
distance_matrix = Utilities.distance_matrix(ridb_slice,'FacilityLatitude','FacilityLongitude')

In [65]:
distance_matrix

FacilityName,CAMP CREEK,Lower Camp Creek Campground,Badger Lake Campground,Eagle Creek Campground,EAGLE CREEK CAMPGROUND
FacilityName,,,,,
CAMP CREEK,0.000000,156.374069,15.076343,23.608398,23.606868
Lower Camp Creek Campground,156.374069,0.000000,141.917916,166.738720,166.737864
Badger Lake Campground,15.076343,141.917916,0.000000,29.420905,29.419532
Eagle Creek Campground,23.608398,166.738720,29.420905,0.000000,0.001534
EAGLE CREEK CAMPGROUND,23.606868,166.737864,29.419532,0.001534,0.000000


In [66]:
matches = np.where((np.triu(distance_matrix) < 0.1) & (np.triu(distance_matrix) > 0))

In [67]:
matches

(array([3]), array([4]))

In [68]:
%autoreload 2

In [83]:
df_dedupe = Utilities.dedupe_by_distance(ridb_slice,0.1,'FacilityLatitude','FacilityLongitude','LastUpdatedDate')

drop least recent
16    EAGLE  CREEK  CAMPGROUND
Name: FacilityName, dtype: object


In [84]:
df_dedupe

,FacilityName,FacilityLatitude,FacilityLongitude,LastUpdatedDate
12,CAMP CREEK,45.3031,-121.865,2016-05-12
13,Lower Camp Creek Campground,44.6725,-118.8,2016-05-09
14,Badger Lake Campground,45.305,-121.555,2016-05-09
15,Eagle Creek Campground,45.6422,-121.925,2016-05-09


In [34]:
for pair in zip(matches[0],matches[1]):
    pair_sub = web_data.loc[np.array(pair)]
    print(pair_sub.LastUpdatedDate)
    print(pair_sub.index[0])
    least_recent = pair_sub[pair_sub['LastUpdatedDate'] != max(pair_sub['LastUpdatedDate'])]
    #web_data_copy = web_data_copy.drop(least_recent.index)
    web_data_copy = web_data_copy.drop(pair_sub.index[0])

3    2016-05-09
5    2015-10-15
Name: LastUpdatedDate, dtype: object
3
15    2016-05-09
16    2015-12-03
Name: LastUpdatedDate, dtype: object
15


In [35]:
web_data_copy.index

Int64Index([0, 1, 2, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype='int64')

In [17]:
pd.DataFrame(distance_matrix)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.000000,100.900530,46.163737,12.976290,64.103978,12.175875,19.562382,17.211835,25.018834,18.612140,19.438193,7733.425088,6.979180,163.073510,21.303609,19.983981,19.982539
1,100.900530,0.000000,83.734564,100.938215,78.025573,100.569274,89.491834,93.540246,81.788054,82.353279,81.909957,7682.877439,93.943853,76.959602,82.420492,111.547075,111.545795
2,46.163737,83.734564,0.000000,56.930092,18.479713,56.074328,26.765804,55.780765,52.768527,37.364158,34.974674,7753.217640,43.073363,158.121638,48.073295,66.038563,66.037081
3,12.976290,100.938215,56.930092,0.000000,74.142274,0.865389,30.285620,8.011761,19.666662,22.837837,24.750991,7721.789836,13.856812,158.263553,18.552143,11.104309,11.102843
4,64.103978,78.025573,18.479713,74.142274,0.000000,73.304475,44.545719,72.043606,67.172547,52.990975,50.657334,7756.047360,60.382097,154.601457,62.959324,83.763191,83.761685
5,12.175875,100.569274,56.074328,0.865389,73.304475,0.000000,29.423587,8.001545,19.480829,22.116835,24.001729,7722.370359,13.001124,158.235513,18.153645,11.717293,11.715805
6,19.562382,89.491834,26.765804,30.285620,44.545719,29.423587,0.000000,30.126416,30.394952,14.742527,12.888113,7740.053339,16.472163,157.473200,25.293603,39.299137,39.297646
7,17.211835,93.540246,55.780765,8.011761,72.043606,8.001545,30.126416,0.000000,11.852961,19.152904,21.426614,7716.213254,14.480998,150.255771,11.772738,18.006988,18.005711
8,25.018834,81.788054,52.768527,19.666662,67.172547,19.480829,30.394952,11.852961,0.000000,15.987911,18.262591,7710.712945,19.366320,139.056452,5.101925,29.810716,29.809480
9,18.612140,82.353279,37.364158,22.837837,52.990975,22.116835,14.742527,19.152904,15.987911,0.000000,2.389617,7725.400631,11.762580,146.369798,10.976231,33.735543,33.734024


In [1]:
from classes.RidbData import RidbData

In [2]:
dict(apiKey = "somestring").update(dict(activity_id=9))


In [3]:
ridb_campgrounds = RidbData("ridb_campgrounds","facilities",dict(activity_id=9, state='OR'))

{'state': 'OR', 'apiKey': '5CCBD8E92D5B4F238F8BFC2F579E2EC7', 'activity_id': 9}


In [4]:
ridb_campgrounds.extract()

url: https://ridb.recreation.gov/api/v1/facilities
params: {'state': 'OR', 'apiKey': '5CCBD8E92D5B4F238F8BFC2F579E2EC7', 'activity_id': 9}


In [7]:
ridb_campgrounds.df.head()

,FacilityAdaAccess,FacilityDescription,FacilityDirections,FacilityEmail,FacilityID,FacilityLatitude,FacilityLongitude,FacilityMapURL,FacilityName,FacilityPhone,FacilityReservationURL,FacilityTypeDescription,FacilityUseFeeDescription,GEOJSON.COORDINATES,GEOJSON.TYPE,Keywords,LastUpdatedDate,LegacyFacilityID,OrgFacilityID,StayLimit
0,ADA Access,This Watchable Wildlife site is located at the...,Silver Creek Landing is located on the McKenzi...,BLM_OR_EU_Mail@blm.gov,201777,44.065900,-122.263400,,Silver Creek Landing,541-683-6600,,,,"[-122.2634, 44.0659]",Point,,2015-07-30,BLM,,
1,,"Facilities include paved parking, a trail, a c...",The Wood River Wetland is located about 30 mil...,BLM_OR_LV_Mailbox@blm.gov,201783,42.582720,-121.969870,,Wood River Wetland,541-947-2177,,,,"[-121.96987, 42.58272]",Point,,2012-06-02,BLM,,
2,,The Warm Springs National Fish Hatchery (NFH) ...,The hatchery is 5 miles northwest of Kah-Nee-T...,Mike_Paiya@fws.gov,203854,44.866699,-121.250000,,Warm Springs National Fish Hatchery (Oregon),(541) 553-1692,,,,"[-121.25, 44.86669922]",Point,,2013-01-10,FWS,,
3,,"At the Eagle Creek National Fish Hatchery, egg...",Located along Eagle Creek 13 miles E of Estaca...,Douglas_Dysart@fws.gov,203855,45.276199,-122.205002,,Eagle Creek National Fish Hatchery,(503) 630-6270,,,,"[-122.20500183, 45.27619934]",Point,,2013-01-02,FWS,,
4,ADA Access:,This boat launch site is one of many that are ...,This site is located on the McKenzie River eas...,BLM_OR_EU_Mail@blm.gov,210601,44.072800,-122.281000,,Rennie Landing: McKenzie River,(541) 683-6600,,Boat Launch,,"[-122.281, 44.0728]",Point,,2011-08-02,BLM,,


In [8]:
ridb_activities = RidbData("ridb_activities","activities",dict())

{'apiKey': '5CCBD8E92D5B4F238F8BFC2F579E2EC7'}


In [9]:
ridb_activities.extract()

url: https://ridb.recreation.gov/api/v1/activities
params: {'apiKey': '5CCBD8E92D5B4F238F8BFC2F579E2EC7'}


In [16]:
ridb_hiking = RidbData("ridb_hiking", "facilities", dict(activity_id=14, state='OR'))

{'state': 'OR', 'apiKey': '5CCBD8E92D5B4F238F8BFC2F579E2EC7', 'activity_id': 14}


In [17]:
ridb_hiking.extract()

url: https://ridb.recreation.gov/api/v1/facilities
params: {'state': 'OR', 'apiKey': '5CCBD8E92D5B4F238F8BFC2F579E2EC7', 'activity_id': 14}


In [18]:
ridb_hiking.df

,FacilityAdaAccess,FacilityDescription,FacilityDirections,FacilityEmail,FacilityID,FacilityLatitude,FacilityLongitude,FacilityMapURL,FacilityName,FacilityPhone,FacilityReservationURL,FacilityTypeDescription,FacilityUseFeeDescription,GEOJSON.COORDINATES,GEOJSON.TYPE,Keywords,LastUpdatedDate,LegacyFacilityID,OrgFacilityID,StayLimit
0,ADA Access,This Watchable Wildlife site is located at the...,Silver Creek Landing is located on the McKenzi...,BLM_OR_EU_Mail@blm.gov,201777,44.065900,-122.263400,,Silver Creek Landing,541-683-6600,,,,"[-122.2634, 44.0659]",Point,,2015-07-30,BLM,,
1,,"Facilities include paved parking, a trail, a c...",The Wood River Wetland is located about 30 mil...,BLM_OR_LV_Mailbox@blm.gov,201783,42.582720,-121.969870,,Wood River Wetland,541-947-2177,,,,"[-121.96987, 42.58272]",Point,,2012-06-02,BLM,,
2,,The Warm Springs National Fish Hatchery (NFH) ...,The hatchery is 5 miles northwest of Kah-Nee-T...,Mike_Paiya@fws.gov,203854,44.866699,-121.250000,,Warm Springs National Fish Hatchery (Oregon),(541) 553-1692,,,,"[-121.25, 44.86669922]",Point,,2013-01-10,FWS,,
3,,"At the Eagle Creek National Fish Hatchery, egg...",Located along Eagle Creek 13 miles E of Estaca...,Douglas_Dysart@fws.gov,203855,45.276199,-122.205002,,Eagle Creek National Fish Hatchery,(503) 630-6270,,,,"[-122.20500183, 45.27619934]",Point,,2013-01-02,FWS,,
4,ADA Access:,This boat launch site is one of many that are ...,This site is located on the McKenzie River eas...,BLM_OR_EU_Mail@blm.gov,210601,44.072800,-122.281000,,Rennie Landing: McKenzie River,(541) 683-6600,,Boat Launch,,"[-122.281, 44.0728]",Point,,2011-08-02,BLM,,
5,ADA Access,This hidden site is utilized by those who like...,This site is located on the McKenzie River eas...,BLM_OR_EU_Mail@blm.gov,210602,44.075650,-122.743100,,Taylor Landing: McKenzie River,(541) 683-6600,,Boat Launch,,"[-122.7431, 44.07565]",Point,,2012-06-02,BLM,,
6,"The pavilion, eight of 11 campsites, and all r...","The campground has 10 campsites, a 50-person p...","12 miles west of Sutherlin on Highway 138, cro...",BLM_OR_RB_Mail@blm.gov,210606,43.498013,-123.493538,http://www.or.blm.gov/roseburg/rec/maps/Eaglev...,Eagleview Group Campground,541-440-4930,,Campground,$130 per day plus $200 deposit,"[-123.493538, 43.498013]",Point,,2012-06-02,,,
7,,Lone Pine group site is an entire campground l...,22 miles east of Roseburg (6 miles east of Gli...,BLM_OR_RB_Mail@blm.gov,210607,44.777670,-119.623700,http://www.or.blm.gov/roseburg/rec/maps/LonePi...,Lone Pine Group Campground,541-440-4930,,Campground,$130 per day plus $200 deposit,"[-119.6237, 44.77767]",Point,,2012-06-02,,,
8,,Tyee Recreation Site has 15 first come campsit...,"12 miles west of Sutherlin on Highway 138, cro...",BLM_OR_RB_Mail@blm.gov,224333,43.484985,-123.484100,http://maps.google.com/maps?f=q&hl=en&geocode...,Tyee Recreation Site,541-440-4930,,,"$10 per night, $4 extra vehicle. Pavilion us...","[-123.4841, 43.484985]",Point,,2011-08-02,372045,,
9,,"<p>The Ashland Springs Hotel, built in 1925 an...","<p>In downtown Ashland, just steps from\r\nthe...",info@historichotels.org,231772,42.195910,-122.712600,,Ashland Springs Hotel,+1 800 678 8946,,,,"[-122.7126, 42.19591]",Point,,2015-08-12,,,


In [36]:
testd = dict(activity_id=9, state='OR')

In [37]:
testd['spam']

KeyError: 'spam'